# Bibliotecas

In [ ]:
# ! pip install selenium --quiet
# ! apt-get update --quiet
# ! apt install chromium-chromedriver --quiet

In [ ]:
# from    selenium import webdriver
import  pandas as pd
# from  selenium import webdriver
import  numpy as np
import  re
import  plotly.express as px
from    plotly.subplots import make_subplots
import  plotly.graph_objects as go
from    sklearn.decomposition import PCA
from    sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Webdriver

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# Nikola Jokic

In [ ]:
# wd = webdriver.Chrome(options=options)

# temporadas_jokic = np.arange(2016, 2024)

# lista_dfs = []

# # Coleta os dados do jogador de 2016 a 2023
# for ano in temporadas_jokic:
#   site = 'https://www.espn.com.br/nba/jogador/jogo-a-jogo/_/id/3112335/tipo/nba/ano/' + str(ano)

#   wd.get(site)

#   html = wd.page_source

#   scrapping = pd.read_html(html)

#   df_stats_ano = pd.concat([scrapping[0],
#                             scrapping[1],
#                             scrapping[2],
#                             scrapping[3],
#                             scrapping[4],
#                             scrapping[5],
#                             scrapping[6]])

#   df_stats_ano.insert(0, 'Temporada', ano)

#   lista_dfs.append(df_stats_ano)

#   df_stats_jokic = pd.concat(lista_dfs).reset_index(drop=True)

In [ ]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset raw from csv

In [ ]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
                             encoding='utf-8',
                             sep=',')

In [ ]:
df_stats_jokic.reset_index(drop=True, inplace=True)

In [ ]:
df_stats_jokic.head(6)

,Temporada,Date,ADV,Resultado,MIN,FG,FG%,3PT,3P%,FT,FT%,REB,V,BLK,STL,PF,TO,PTS,Estat. Temporada Regular
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,NaN
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,NaN
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,NaN
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,NaN
5,2016,april,april,april,30.8,4.4-7.8,56.4,0.4-2.0,20.0,2.4-2.6,92.3,10.8,4.0,1.2,0.8,2.6,0.6,11.6,NaN


# Data wrangling

In [ ]:
df_stats_jokic.isnull().sum()

Temporada                     0
Date                          2
ADV                           2
Resultado                     2
MIN                           0
FG                            0
FG%                           0
3PT                           0
3P%                           0
FT                            0
FT%                           0
REB                           0
V                             0
BLK                           0
STL                           0
PF                            0
TO                            0
PTS                           0
Estat. Temporada Regular    654
dtype: int64

In [ ]:
df_stats_jokic.drop(columns='Estat. Temporada Regular', inplace=True)

df_stats_jokic.dropna(axis=0, how='any', inplace=True)

In [ ]:
df_stats_jokic.isnull().sum()

Temporada    0
Date         0
ADV          0
Resultado    0
MIN          0
FG           0
FG%          0
3PT          0
3P%          0
FT           0
FT%          0
REB          0
V            0
BLK          0
STL          0
PF           0
TO           0
PTS          0
dtype: int64

In [ ]:
renamed_columns = {'Date': 'Data',
                   'V': 'Assistencias',
                   'PF': 'Faltas',
                   'TO': 'Tunovers',
                   'MIN': 'Minutos',
                   'REB': 'Rebotes',
                   'BLK': 'Bloqueios',
                   'STL': 'Desarmes',
                   'PTS': 'Pontos'}

df_stats_jokic.rename(columns=renamed_columns, inplace=True)

In [ ]:
df_stats_jokic.head(6)

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0
5,2016,april,april,april,30.8,4.4-7.8,56.4,0.4-2.0,20.0,2.4-2.6,92.3,10.8,4.0,1.2,0.8,2.6,0.6,11.6


In [ ]:
# Remoção de linhas com texto de nome de meses
df_stats_jokic = df_stats_jokic[(df_stats_jokic['Data']   != 'january')
                                & (df_stats_jokic['Data'] != 'february')
                                & (df_stats_jokic['Data'] != 'march')
                                & (df_stats_jokic['Data'] != 'april')
                                & (df_stats_jokic['Data'] != 'may')
                                & (df_stats_jokic['Data'] != 'june')
                                & (df_stats_jokic['Data'] != 'july')
                                & (df_stats_jokic['Data'] != 'august')
                                & (df_stats_jokic['Data'] != 'september')
                                & (df_stats_jokic['Data'] != 'october')
                                & (df_stats_jokic['Data'] != 'november')
                                & (df_stats_jokic['Data'] != 'december')]

df_stats_jokic.reset_index(drop=True, inplace=True)

In [ ]:
# Verifica a remoção de linhas com texto de nome de meses
df_stats_jokic[(df_stats_jokic['Data']   == 'january')
               | (df_stats_jokic['Data'] == 'february')
               | (df_stats_jokic['Data'] == 'march')
               | (df_stats_jokic['Data'] == 'april')
               | (df_stats_jokic['Data'] == 'may')
               | (df_stats_jokic['Data'] == 'june')
               | (df_stats_jokic['Data'] == 'july')
               | (df_stats_jokic['Data'] == 'august')
               | (df_stats_jokic['Data'] == 'september')
               | (df_stats_jokic['Data'] == 'october')
               | (df_stats_jokic['Data'] == 'november')
               | (df_stats_jokic['Data'] == 'december')]

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos


In [ ]:
# Regex de informações das colunas para novas colunas exclusivas
df_stats_jokic['FG_Tentativas'] = df_stats_jokic['FG'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FG_Acertos'] = df_stats_jokic['FG'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['3PT_Tentativas'] = df_stats_jokic['3PT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['3PT_Acertos'] = df_stats_jokic['3PT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['FT_Tentativas'] = df_stats_jokic['FT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FT_Acertos'] = df_stats_jokic['FT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['Adversario'] = df_stats_jokic['ADV'].str.extract(r'(?:em|x)([A-Z]+)')  # (?:em|x) - non-capturing group that matches 'em' or 'x'

df_stats_jokic['Placar'] = df_stats_jokic['Resultado'].str.extract(r'(\d+-\d+)')

df_stats_jokic['V_D'] = df_stats_jokic['Resultado'].str.extract(r'(^D|V)')

df_stats_jokic['Casa_Fora'] = df_stats_jokic['ADV'].str.extract(r'(em|x)')

df_stats_jokic['Dia_semana'] = df_stats_jokic['Data'].str.extract(r'([A-Za-z0-9_À-ÿ]+)')

df_stats_jokic['Mes'] = df_stats_jokic['Data'].str.extract(r'\b\d+/\b(\d+)')

<ipython-input-566-4812f5a36f9a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-566-4812f5a36f9a>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-566-4812f5a36f9a>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-566-481

In [ ]:
df_stats_jokic[(df_stats_jokic['Adversario'] == 'LEB')
               | (df_stats_jokic['Adversario'] == 'GIA')
               | (df_stats_jokic['Adversario'] == 'DUR')]  # Participações no All-Star

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,V_D,Casa_Fora,Dia_semana,Mes
252,2019,Seg 18/2,xLEB*,D178-164,13.0,3-4,75.0,0-0,0.0,0-0,0.0,9.0,0.0,0.0,0.0,1.0,1.0,6.0,4,3,0,0,0,0,LEB,178-164,D,x,Seg,2
327,2020,Seg 17/2,xGIA*,V157-155,12.0,2-4,50.0,1-3,33.3,0-0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,5.0,4,2,3,1,0,0,GIA,157-155,V,x,Seg,2
419,2021,Seg 8/3,xDUR*,V170-150,19.0,3-4,75.0,0-0,0.0,0-0,0.0,7.0,5.0,1.0,0.0,1.0,2.0,6.0,4,3,0,0,0,0,DUR,170-150,V,x,Seg,3
478,2022,Seg 21/2,xDUR*,V163-160,21.0,4-5,80.0,2-3,66.7,0-0,0.0,9.0,8.0,0.0,1.0,0.0,1.0,10.0,5,4,3,2,0,0,DUR,163-160,V,x,Seg,2
549,2023,Seg 20/2,xGIA*,D184-175,20.0,2-4,50.0,0-1,0.0,0-0,0.0,5.0,6.0,0.0,2.0,0.0,2.0,4.0,4,2,1,0,0,0,GIA,184-175,D,x,Seg,2


In [ ]:
df_stats_jokic = df_stats_jokic.replace('^LEB$', 'ALL', regex=True)
df_stats_jokic = df_stats_jokic.replace('^GIA$', 'ALL', regex=True)
df_stats_jokic = df_stats_jokic.replace('^DUR$', 'ALL', regex=True)

In [ ]:
df_stats_jokic['Dia_semana'] = (df_stats_jokic['Dia_semana'].map({'Seg': 0,
                                                                  'Ter': 1,
                                                                  'Qua': 2,
                                                                  'Qui': 3,
                                                                  'Sex': 4,
                                                                  'Sáb': 5,
                                                                  'Dom': 6}))  # Dias da semana no padrão python

In [ ]:
pd.set_option('display.max_columns', None)

df_stats_jokic['%FG'] = round(df_stats_jokic['FG_Acertos'] / df_stats_jokic['FG_Tentativas'], 2)

df_stats_jokic['%3PT'] = round(df_stats_jokic['3PT_Acertos'] / df_stats_jokic['3PT_Tentativas'], 2)

df_stats_jokic['%FT'] = round(df_stats_jokic['FT_Acertos'] / df_stats_jokic['FT_Tentativas'], 2)

df_stats_jokic['Casa_Fora'] = df_stats_jokic['Casa_Fora'].map({'em': 'fora','x': 'casa'})

## Análise do placar

In [ ]:
# Cria colunas nos respectivos indexes
df_stats_jokic.insert(26, 'Placar_Denver', '')
df_stats_jokic.insert(27, 'Placar_Adversario', '')
df_stats_jokic.insert(28, 'Placar_Vencedor', '')
df_stats_jokic.insert(29, 'Placar_Perdedor', '')

In [ ]:
df_stats_jokic['Placar_Vencedor'] = df_stats_jokic['Placar'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['Placar_Perdedor'] = df_stats_jokic['Placar'].str.extract(r'-(\d+)').astype(int)

In [ ]:
"""
Identifica se o jogo foi vitória ou derrota.

O padrão do placar é sempre o nº de pontos do vencedor na frente.

Se o Denver ganhou, atribui o Placar_Vencedor para a coluna Placar_Denver
e Placar_Perdedor para Placar_Adversario.

Se o Denver perdeu é realizado o processo oposto.
"""
for i_resultado, resultado in enumerate(df_stats_jokic['V_D']):

    if resultado == 'D':
        df_stats_jokic['Placar_Denver'].loc[i_resultado] = df_stats_jokic['Placar_Perdedor'].loc[i_resultado]
        df_stats_jokic['Placar_Adversario'].loc[i_resultado] = df_stats_jokic['Placar_Vencedor'].loc[i_resultado]

    else:
        df_stats_jokic['Placar_Denver'].loc[i_resultado] = df_stats_jokic['Placar_Vencedor'].loc[i_resultado]
        df_stats_jokic['Placar_Adversario'].loc[i_resultado] = df_stats_jokic['Placar_Perdedor'].loc[i_resultado]

<ipython-input-573-38d356b59467>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-573-38d356b59467>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-573-38d356b59467>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-573-38d356b59467>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
df_stats_jokic.head()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,Placar_Denver,Placar_Adversario,Placar_Vencedor,Placar_Perdedor,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,107-99,99,107,107,99,D,fora,3,4,0.70,0.50,1.00
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,100-84,84,100,100,84,D,casa,6,4,0.78,0.00,0.83
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,102-98,102,98,102,98,V,casa,5,4,0.33,0.00,1.00
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,124-102,102,124,124,102,D,casa,2,4,0.60,0.33,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,115-106,106,115,115,106,D,casa,6,4,0.33,0.00,1.00


## Identificando as conferências dos times

In [ ]:
conferencia_leste = ['ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND', 'MIA', 'MIL', 'NY', 'ORL', 'PHI', 'TOR', 'WSH']
conferencia_oeste = ['DAL', 'DEN', 'GS', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN', 'NO', 'OKC', 'PHX', 'POR', 'SAC', 'SA', 'UTAH']

In [ ]:
df_stats_jokic.insert(25, 'Conferencia', '')  # Cria a coluna Liga no index 16

In [ ]:
def conferencia(time):
    """
    Identifica a qual liga o time pertence
    """
    if time in conferencia_leste:
        return 'Leste'
    elif time in conferencia_oeste:
        return 'Oeste'
    else:
        return '-'

In [ ]:
# series.apply aplica a função sobre a série Adversario
# Depois a respectiva liga é inserida na coluna Conferencia
df_stats_jokic['Conferencia'] = df_stats_jokic['Adversario'].apply(conferencia)

In [ ]:
df_stats_jokic.drop(columns=['Data', 'ADV', 'Resultado', 'FG', 'FG%', '3PT',
                             '3P%', 'FT', 'FT%', 'Placar', 'Placar_Vencedor',
                             'Placar_Perdedor'], inplace=True)

df_stats_jokic.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Conferencia,Placar_Denver,Placar_Adversario,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,Oeste,99,107,D,fora,3,4,0.70,0.50,1.00
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,Oeste,84,100,D,casa,6,4,0.78,0.00,0.83
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,Oeste,102,98,V,casa,5,4,0.33,0.00,1.00
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,Oeste,102,124,D,casa,2,4,0.60,0.33,NaN
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,Oeste,106,115,D,casa,6,4,0.33,0.00,1.00


## Salva dataset ajustado

In [ ]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset from csv

In [ ]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
                             encoding='utf-8',
                             sep=',')

df_stats_jokic = df_stats_jokic[df_stats_jokic['Adversario'] != 'ALL'].copy()  # Remova as linhas com dados do All-Star

In [ ]:
df_stats_jokic['V_D'].value_counts()

V    351
D    245
Name: V_D, dtype: int64

# Análise gráfica

## %stats

In [ ]:
media_porcent_temporadas = df_stats_jokic[['Temporada', '%FG', '%FT', '%3PT']].groupby('Temporada').mean()

In [ ]:
fig = px.line(media_porcent_temporadas,
              x=media_porcent_temporadas.index,
              y=['%FG', '%FT', '%3PT'],
              labels={'variable': ''})

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic %stats',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)
fig.update_yaxes(title_text=None)
fig.show()

## Pontos

In [ ]:
media_pontos_temporadas = df_stats_jokic[['Temporada', 'Pontos']].groupby('Temporada').mean()

media_pontos_temporadas_por_tipo = df_stats_jokic[['Temporada', 'FG_Acertos', 'FT_Acertos', '3PT_Acertos']].groupby('Temporada').mean()

In [ ]:
media_pontos_temporadas_por_tipo['FG_Pontos'] = media_pontos_temporadas_por_tipo['FG_Acertos'] * 2

media_pontos_temporadas_por_tipo['FT_Pontos'] = media_pontos_temporadas_por_tipo['FT_Acertos'] * 1

media_pontos_temporadas_por_tipo['3_Pontos'] = media_pontos_temporadas_por_tipo['3PT_Acertos'] * 3

media_pontos_temporadas_por_tipo

,FG_Acertos,FT_Acertos,3PT_Acertos,FG_Pontos,FT_Pontos,3_Pontos
Temporada,,,,,,
2016,3.837500,1.925000,0.350000,7.675000,1.925000,1.050000
2017,6.767123,2.575342,0.616438,13.534247,2.575342,1.849315
2018,6.720000,3.546667,1.480000,13.440000,3.546667,4.440000
2019,7.700000,3.612500,1.037500,15.400000,3.612500,3.112500
2020,7.739726,3.369863,1.095890,15.479452,3.369863,3.287671
2021,10.166667,4.750000,1.277778,20.333333,4.750000,3.833333
2022,10.324324,5.121622,1.310811,20.648649,5.121622,3.932432
2023,9.362319,4.942029,0.826087,18.724638,4.942029,2.478261


In [ ]:
fig = make_subplots(shared_xaxes=True)

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FG_Pontos'],
                         mode='lines',
                         line=dict(color='#845EC2'),
                         name='FG'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FT_Pontos'],
                         mode='lines',
                         line=dict(color='#008BC9'),
                         name='FT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['3_Pontos'],
                         mode='lines',
                         line=dict(color='#FF8066'),
                         name='3PT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas.index,
                         y=media_pontos_temporadas['Pontos'],
                         mode='lines',
                         line=dict(color='#936C00'),
                         name='Média geral'))

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic - média de pontos nas temporadas',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)

fig.update_yaxes(title_text=None)

fig.show()

## Correlação

In [ ]:
correlacao = df_stats_jokic[['Temporada',
                             'Minutos',
                             'Rebotes',
                             'Assistencias',
                             'Bloqueios',
                             'Desarmes',
                             'Faltas',
                             'Tunovers',
                             'Pontos',
                             'FG_Tentativas',
                             'FG_Acertos',
                             '3PT_Tentativas',
                             '3PT_Acertos',
                             'FT_Tentativas',
                             'FT_Acertos',
                             'Dia_semana',
                             'Mes',
                             '%FG',
                             '%3PT',
                             '%FT']].corr().round(2)

fig = px.imshow(correlacao, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.update_xaxes(side='top')

fig.show()

±0.9 → correlação muito forte

±0.7 a ±0.9 → correlação forte

±0.5 a ±0.7 → correlação moderada

±0.3 a ±0.5 → correlação fraca

### Correlação com pontos

#### Correlação muito forte

In [ ]:
correlacao.loc[correlacao['Pontos'] >= 0.9].index

Index(['Pontos', 'FG_Acertos'], dtype='object')

#### Correlação forte

In [ ]:
correlacao.loc[(correlacao['Pontos'] >= 0.7)
               & (correlacao['Pontos'] < 0.9)].index

Index(['FG_Tentativas'], dtype='object')

#### Correlação moderada

In [ ]:
correlacao.loc[(correlacao['Pontos'] >= 0.5)
               & (correlacao['Pontos'] < 0.7)].index

Index(['Temporada', 'Minutos', 'FT_Tentativas', 'FT_Acertos'], dtype='object')

#### Correlação fraca

In [ ]:
correlacao.loc[(correlacao['Pontos'] >= 0.3)
               & (correlacao['Pontos'] < 0.5)].index

Index(['Rebotes', 'Assistencias', '3PT_Tentativas', '3PT_Acertos', '%FG'], dtype='object')

### Conclusões

#### Pontuação

Os pontos do Jokic tem correlação:
* Muito forte com lances de quadra (FG)
* Moderada com lances livres (FT)
* Fraca com lances de 3 pontos (3PT)

In [ ]:
media_acertos_fg = df_stats_jokic['FG_Acertos'].mean().round(2)
media_acertos_ft = df_stats_jokic['FT_Acertos'].mean().round(2)
media_acertos_3pt = df_stats_jokic['3PT_Acertos'].mean().round(2)

print('Média de acertos FG por jogo:', media_acertos_fg)
print('Média de acertos FT por jogo:', media_acertos_ft)
print('Média de acertos 3PT por jogo:', media_acertos_3pt)

Média de acertos FG por jogo: 7.77
Média de acertos FT por jogo: 3.7
Média de acertos 3PT por jogo: 0.99


#### Aproveitamento

In [ ]:
media_aproveitamento_porcent_fg = (df_stats_jokic['%FG'].mean() * 100).round(2)
media_aproveitamento_porcent_ft = (df_stats_jokic['%FT'].mean() * 100).round(2)
media_aproveitamento_porcent_3pt = (df_stats_jokic['%3PT'].mean() * 100).round(2)

print('Média de aproveitamento FG por jogo:', media_aproveitamento_porcent_fg, '%')
print('Média de aproveitamento FT por jogo:', media_aproveitamento_porcent_ft, '%')
print('Média de aproveitamento 3PT por jogo:', media_aproveitamento_porcent_3pt, '%')

Média de aproveitamento FG por jogo: 55.69 %
Média de aproveitamento FT por jogo: 82.0 %
Média de aproveitamento 3PT por jogo: 35.07 %


A maior porcentagem de aproveitamento é de lances livres (FT)

Apesar do bom aproveitamento, a correlação é mais forte com FG por valer mais pontos que FT

#### Média de pontos

In [ ]:
pontos_fg = 2
pontos_ft = 1
pontos_3pt = 3

media_pontos_fg = (df_stats_jokic['FG_Acertos'].mean() * pontos_fg).round(2)
media_pontos_ft = (df_stats_jokic['FT_Acertos'].mean() * pontos_ft).round(2)
media_pontos_3pt = (df_stats_jokic['3PT_Acertos'].mean() * pontos_3pt).round(2)
media_pontos_por_jogo = (df_stats_jokic['Pontos'].mean()).round(2)

print('Média de pontos FG:', media_pontos_fg)
print('Média de pontos FT:', media_pontos_ft)
print('Média de pontos 3PT:', media_pontos_3pt)
print('Média de pontos por jogo:', media_pontos_por_jogo)

Média de pontos FG: 15.53
Média de pontos FT: 3.7
Média de pontos 3PT: 2.98
Média de pontos por jogo: 20.22


# Substituição de variáveis categóricas

Objetivo
* Identificar quais componentes são essenciais para estimar o nº de pontos

In [ ]:
df_stats_jokic_ohe = df_stats_jokic.copy()

## One Hot Encoder

### Adversario e mandos de jogo

In [ ]:
nomes_valores_unicos = pd.concat([df_stats_jokic['Adversario'], df_stats_jokic['Casa_Fora']]).unique()

nomes_colunas = np.sort(np.array(nomes_valores_unicos))  # Ordem alfabética

nomes_colunas

array(['ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DET', 'GS',
       'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NO', 'NY',
       'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SA', 'SAC', 'TOR', 'UTAH',
       'WSH', 'casa', 'fora'], dtype=object)

In [ ]:
nomes_valores_unicos

array(['POR', 'UTAH', 'SA', 'OKC', 'SAC', 'NO', 'MEM', 'DAL', 'LAC',
       'LAL', 'PHI', 'CLE', 'CHA', 'ATL', 'ORL', 'MIA', 'WSH', 'PHX',
       'NY', 'BKN', 'BOS', 'DET', 'CHI', 'TOR', 'IND', 'GS', 'MIN', 'HOU',
       'MIL', 'fora', 'casa'], dtype=object)

In [ ]:
ohe = OneHotEncoder()

adversario_casa_fora_sem_categoricas = ohe.fit_transform(df_stats_jokic[['Adversario', 'Casa_Fora']]).toarray()

In [ ]:
df_stats_jokic_adv_casa_fora_ohe = pd.DataFrame(adversario_casa_fora_sem_categoricas, columns=nomes_colunas)

In [ ]:
df_stats_jokic_adv_casa_fora_ohe

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,GS,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Vitórias e derrotas

In [ ]:
vitoria_derrota_sem_repeticao = df_stats_jokic['V_D'].unique()

vitoria_derrota_colunas = np.sort(np.array(vitoria_derrota_sem_repeticao))  # Ordem alfabética

vitoria_derrota_colunas

array(['D', 'V'], dtype=object)

In [ ]:
vitoria_derrota_sem_categoricas = ohe.fit_transform(df_stats_jokic[['V_D']]).toarray()

In [ ]:
df_stats_jokic_vitoria_derrota_ohe = pd.DataFrame(vitoria_derrota_sem_categoricas, columns=vitoria_derrota_colunas)

In [ ]:
df_stats_jokic_vitoria_derrota_ohe

,D,V
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
591,0.0,1.0
592,1.0,0.0
593,0.0,1.0
594,0.0,1.0


### Conferências

In [ ]:
conferencias_sem_repeticao = df_stats_jokic['Conferencia'].unique()

conferencias_colunas = np.sort(np.array(conferencias_sem_repeticao))  # Ordem alfabética

conferencias_colunas

array(['Leste', 'Oeste'], dtype=object)

In [ ]:
conferencias_sem_categoricas = ohe.fit_transform(df_stats_jokic[['Conferencia']]).toarray()

In [ ]:
df_stats_jokic_conferencias_ohe = pd.DataFrame(conferencias_sem_categoricas, columns=conferencias_colunas)

In [ ]:
df_stats_jokic_conferencias_ohe

,Leste,Oeste
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
591,0.0,1.0
592,0.0,1.0
593,0.0,1.0
594,0.0,1.0


In [ ]:
df_stats_jokic_sem_categoricas = pd.concat([df_stats_jokic_adv_casa_fora_ohe,
                                            df_stats_jokic_vitoria_derrota_ohe,
                                            df_stats_jokic_conferencias_ohe],
                                           axis=1)

df_stats_jokic_sem_categoricas

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,GS,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V,Leste,Oeste
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [ ]:
df_stats_jokic_ohe = pd.concat([df_stats_jokic_ohe, df_stats_jokic_sem_categoricas], axis=1)

df_stats_jokic_ohe.drop(columns=['Adversario', 'Casa_Fora', 'V_D', 'Conferencia'], inplace=True)

In [ ]:
df_stats_jokic_ohe.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Placar_Denver,Placar_Adversario,Dia_semana,Mes,%FG,%3PT,%FT,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,GS,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V,Leste,Oeste
0,2016.0,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10.0,7.0,2.0,1.0,3.0,3.0,99.0,107.0,3.0,4.0,0.70,0.50,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,2016.0,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9.0,7.0,2.0,0.0,6.0,5.0,84.0,100.0,6.0,4.0,0.78,0.00,0.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,2016.0,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9.0,3.0,2.0,0.0,2.0,2.0,102.0,98.0,5.0,4.0,0.33,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,2016.0,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5.0,3.0,3.0,1.0,0.0,0.0,102.0,124.0,2.0,4.0,0.60,0.33,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,2016.0,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6.0,2.0,1.0,0.0,2.0,2.0,106.0,115.0,6.0,4.0,0.33,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


### Salva csv com One Hot Encoder

In [ ]:
# df_stats_jokic_ohe.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ohe.csv',
#                           encoding='utf-8',
#                           sep=',',
#                           index=False)

## Correlação OHE

In [ ]:
correlacao_ohe = df_stats_jokic_ohe.corr().round(2)

fig = px.imshow(correlacao_ohe, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1800,
                  height=1800)

fig.update_xaxes(side='top')

fig.show()

In [ ]:
df_correlacao_ohe = pd.DataFrame(correlacao_ohe)

df_correlacao_ohe.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Placar_Denver,Placar_Adversario,Dia_semana,Mes,%FG,%3PT,%FT,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,GS,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V,Leste,Oeste
Temporada,1.00,0.48,0.35,0.56,0.01,0.13,-0.05,0.35,0.51,0.43,0.49,0.22,0.16,0.36,0.35,0.29,0.14,-0.04,-0.03,0.20,0.01,0.03,0.01,0.01,0.00,0.01,-0.02,0.01,-0.03,0.01,-0.02,0.01,-0.02,0.01,0.02,0.01,0.01,-0.03,-0.02,0.01,-0.01,-0.00,0.01,0.03,-0.02,-0.01,-0.01,0.00,0.00,-0.00,0.02,-0.01,0.01,-0.17,0.17,0.02,-0.02
Minutos,0.48,1.00,0.57,0.51,0.19,0.22,0.10,0.38,0.67,0.69,0.64,0.45,0.34,0.42,0.41,0.19,0.16,-0.02,-0.18,0.02,0.03,0.08,-0.00,-0.04,-0.03,0.00,-0.11,0.03,-0.05,-0.01,-0.09,-0.03,0.08,-0.01,0.01,0.02,0.09,0.03,-0.02,-0.08,0.03,0.02,0.03,0.06,0.02,0.05,0.04,-0.02,0.02,-0.04,0.04,0.03,-0.03,-0.15,0.15,0.07,-0.07
Rebotes,0.35,0.57,1.00,0.35,0.13,0.14,0.04,0.27,0.44,0.46,0.42,0.19,0.11,0.31,0.29,0.17,-0.04,0.01,-0.16,0.01,-0.02,0.02,0.03,0.02,0.02,0.07,-0.02,-0.02,0.04,0.00,-0.01,-0.00,0.00,0.01,-0.05,0.05,0.05,0.06,-0.03,-0.02,0.03,-0.03,-0.02,-0.01,-0.02,0.00,-0.05,0.01,0.01,-0.06,-0.02,0.02,-0.02,-0.13,0.13,0.07,-0.07
Assistencias,0.56,0.51,0.35,1.00,0.07,0.10,0.01,0.28,0.32,0.24,0.30,0.09,0.12,0.23,0.22,0.42,0.11,0.01,-0.14,0.18,0.06,0.01,-0.01,-0.05,-0.01,-0.03,0.00,0.02,-0.03,-0.00,-0.04,0.01,-0.01,0.01,0.03,-0.00,0.04,0.05,-0.03,-0.03,-0.01,0.01,0.05,0.03,-0.02,0.06,0.04,-0.01,-0.02,-0.04,-0.00,0.04,-0.04,-0.20,0.20,0.02,-0.02
Bloqueios,0.01,0.19,0.13,0.07,1.00,0.03,0.03,0.05,0.17,0.18,0.16,0.14,0.06,0.12,0.11,0.05,0.05,0.03,-0.05,-0.00,-0.01,-0.02,0.01,0.01,0.01,-0.05,0.02,0.04,-0.06,-0.02,-0.04,0.01,0.01,-0.02,0.02,-0.02,-0.03,0.01,0.04,-0.01,-0.01,0.04,0.04,-0.03,-0.00,0.05,0.01,-0.01,0.05,-0.05,-0.02,0.04,-0.04,-0.02,0.02,0.01,-0.01


In [ ]:
df_correlacao_ohe.loc[(df_correlacao_ohe['Pontos'] > 0.5)
                      & (df_correlacao_ohe['Pontos'] != 1)].index

Index(['Temporada', 'Minutos', 'FG_Tentativas', 'FG_Acertos', 'FT_Tentativas',
       'FT_Acertos'],
      dtype='object')

# Principal Component Analysis (PCA)

In [ ]:
pca = PCA()